<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb ReverseAutoDiff.do.txt  -->

## Reverse-mode automatic differentiation
Deep learning frameworks are built upon the foundation of automatic differentiation (AD). Training deep learning models typically involves gradient-based techniques, with autodiff streamlining the gradient acquisition process, even for large and intricate models. 
The majority of deep learning frameworks utilize 'reverse-mode autodiff' due to its efficiency and precision.

In this module, we will delve into the generalization of the chain rule for AD of any function. This is achieved by understanding that all functions are composed of basic operations such as addition, multiplication, subtraction, and division.

AD relies on the fact that all computer programs, no matter how complicated, use a finite set of elementary (unary or binary, e.g. $sin(\cdot)$, $sqrt(\cdot)$) operations as defined by the programming language. 
The value or function computed by the program is simply a composition of these elementary functions. The partial derivatives of the elementary functions are known, and the overall derivatives can be computed using the chain rule.

**Notice.**

The initial examples and code base are modified from <https://sidsite.com/posts/autodiff/>

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (4, 2)
plt.rcParams['figure.dpi'] = 150

## Total derivative and gradient
The function $f : \mathbb{R}^n \to \mathbb{R}^m$, is totally differentiable at $x\in\mathbb{R}^n$ if there is a linear map $df : \mathbb{R}^n \to \mathbb{R}^m$ that satisfies
\begin{align*}
    f(x + dx) = f(x) + df_{x}(dx) + \epsilon(dx),\; dx\in\mathbb{R}^n
    \\
    \lim_{dx\to0} \frac{\epsilon}{||dx||} = 0
\end{align*}
where $df_{x}$ is the total derivative and defined as the best linear approximation at point $x$.

The total derivative is also written as $df_x(dx)=Df_x(dx)=J_f(x)\times dx$ with the Jacobian $J\in\mathbb{R}^{m\times n}$. When the function $f : \mathbb{R}^n \to \mathbb{R}$ maps to one dimension the gradient denoted as $\nabla f \in \mathbb{R}^n$ is the transposed Jacobian $J_f^T$

There are two modes of AD, forward and reverse. When $m << n$ reverse mode AD is the most efficient and therefore most used with deep learning.

## Chain rule
Let $f: \mathbb{R}^n \to \mathbb{R}$, $g: \mathbb{R}^k \to \mathbb{R}$ and $h: \mathbb{R}^n \to \mathbb{R}^k$ with $x \in \mathbb{R}^n$.

Let $f(x) = g\circ h = g(h(x))$. Then, the chain rule gives a nice relation between the total derivatives the Jacobian and the gradient.

\begin{align*}
    df(x) = dg \circ dh = dg(dh(x))\\
    \nabla f_x = (J_g \times J_h)^T
\end{align*}

## Composite Functions

We can generalize the procedure for computing derivatives if the output is obtained through the evaluation of a composite function $f: \mathbb{R}^n \to \mathbb{R}^m$ composed of $ p $ single functions $f_{j+1}: \mathbb{R}^{k_j} \to \mathbb{R}^{k_{j+1}}, k_p=m,k_1=n$ (which is, in fact, how computer codes are represented):

$$ f = f_p \circ f_{p-1} \circ \cdots \circ f_1(x) $$

In forward-mode AD, we apply the chain rule to each function $ f_j $ and combine them in a recursion relationship. For the adjoint (reverse mode) AD, we construct the adjoint for each function $ f_j $ and combine them in reverse order.

Let us describe the process using matrix notation. If we view the forward-mode AD as the result of the multiplication of a series of Jacobian matrices:

$$ J = J_1 \times J_2 \times \cdots \times J_p $$

where each Jacobian matrix $ J_j $ represents either a subroutine or a single statement, then the adjoint approach can be viewed as a product of adjoint subproblems:

$$ J^T = J_p^T \times J_{p-1}^T \times \cdots \times J_1^T $$

## Example
Let us illustrate this with an example for $ p = 2, m=1 $. The computational flow is described by the following diagram:

$$ x \rightarrow f_1(x) \rightarrow f_2(f_1(x)) \rightarrow y $$

For simplicity, let $ a $ denote the output of $ f_1(x) $ and $ b $ denote the output of $ f_2(f_1(x)) $. The scalar $ y $ is the final output.

With these notations, we have:

$$ x \rightarrow a \rightarrow b \rightarrow y $$

### Forward Mode AD
In the AD literature we use dot to represent the partial wrt the input i.e. $\dot{u}=\frac{\partial u}{\partial x}$.
Applying the tangent linear methodology (essentially differentiation line by line), we have:

\begin{align*}
    \dot{a} &= \frac{\partial a}{\partial x} \dot{x} \\
    \dot{b} &= \frac{\partial b}{\partial a} \dot{a} \\
    \dot{y} &= \frac{\partial y}{\partial b} \dot{b}
\end{align*}

Putting everything together, we get:

$$ \dot{y} = \frac{\partial y}{\partial b} \frac{\partial b}{\partial a} \frac{\partial a}{\partial x} \dot{x} $$

<!-- TODO make a point that you have to do this n times for R^n -->

### Reverse Mode AD
Using notation from AD literature, the adjoint quantities $ \bar{x}, \bar{a}, \bar{b}, \bar{y} $ denote the derivatives of $ y $ with respect to $ x, a, b $ and, respectively, to $ y $. We note that this implies $ \bar{y} = 1 $.

Differentiating again, and with a superscript $ T $ denoting a matrix or vector transpose, we obtain:

\begin{align*}
    \bar{x} &= \left( \frac{\partial y}{\partial x} \right)^T = \left( \frac{\partial y}{\partial a} \frac{\partial a}{\partial x} \right)^T = \left( \frac{\partial a}{\partial x} \right)^T \bar{a} \\
    \bar{a} &= \left( \frac{\partial y}{\partial a} \right)^T = \left( \frac{\partial y}{\partial b} \frac{\partial b}{\partial a} \right)^T = \left( \frac{\partial b}{\partial a} \right)^T \bar{b} \\
    \bar{b} &= \left( \frac{\partial y}{\partial b} \right)^T = \left( \frac{\partial y}{\partial b} \right)^T \cdot 1 = \left( \frac{\partial y}{\partial b} \right)^T \bar{y}
\end{align*}

Putting everything together, we get:

$$ \bar{x} = \left( \frac{\partial a}{\partial x} \right)^T \left( \frac{\partial b}{\partial a} \right)^T \left( \frac{\partial y}{\partial b} \right)^T \bar{y} $$

We notice that the forward mode AD proceeds forward through the process:

$$ \dot{x} \rightarrow \dot{a} \rightarrow \dot{b} \rightarrow \dot{y} $$

while the adjoint (reverse mode) AD proceeds backward through the process:

$$ \bar{x} \leftarrow \bar{a} \leftarrow \bar{b} \leftarrow \bar{xy} $$

In summary, reverse-mode AD involves calculating the adjoints of the output with respect to each intermediate variable by propagating the derivatives backward through each function in the composite function chain. This is efficient when the number of input variables $ n $ is large compared to the number of output variables $ m $.

From now on we will only focus on reverse mode AD


## Small example
We begin with an example where we want to compute the gradient of the function $d: \mathbb{R}^2 \to \mathbb{R}$, $c: \mathbb{R}^2 \to \mathbb{R}$ with respect to $a\in\mathbb{R}$ and $b\in\mathbb{R}$. The point of this exercise is to show that you can use a computational graph directly to state the gradient. Given the function

\begin{align*}
    a &= 4
    \\
    b &= 3
    \\
    c(a,b) &= a + b
    \\
    d(a,b) &= a c(a,b)
\end{align*}

### Adjoint Calculations (Backward pass for a)

1. Initialize the adjoint for the output $ d $:
   $ \bar{d} = 1 $

2. Compute the adjoint for $ c $:

   $ \bar{c} = \bar{d} \cdot \frac{\partial d}{\partial c} $,
   $ \frac{\partial d}{\partial c} = a $,
   $ \bar{c} = 1 \cdot 4 = 4 $

3. Compute the adjoint for $ a $:

   There are two contributions to $\bar{a}$:

   $ \bar{a} = \bar{d} \cdot \frac{\partial d}{\partial a} + \bar{c} \cdot \frac{\partial c}{\partial a} $

   - From $ d $:
     $ \frac{\partial d}{\partial a} = c $
     $ \bar{a}_{(d)} = 1 \cdot 7 = 7 $

   - From $ c $:
     $ \frac{\partial c}{\partial a} = 1 $
     $ \bar{a}_{(c)} = 4 \cdot 1 = 4 $

   Combining both contributions:
   $ \bar{a} = 7 + 4 = 11 $

## Generalized AD
The specific example above shows how to apply backward AD for this particular example. However, we can generalize this process by building a computational graph of the function $d$ shown in the figure below:

<div id="fig:auto-diff-simple-graph"></div>

<img src="figures/auto-diff-simple-graph.png" ><p style="font-size: 0.9em"><i>Figure 1:  The function $d$ is composed of basic operations (addition, multiplication). By representing the function as a graph, we can visualize the flow of information through the function. When computing gradients, we start at the end of the graph and work our way backwards, multiplying the gradients of the children and adding the branches.</i></p>

The function $d$ is composed of basic operations (addition $u_1$, multiplication $u_2$). By representing the function as a graph, we can visualize the flow of information through the function. When computing the gradient, we start at the end of the graph and work our way backwards, multiplying the adjoints of the children and adding the branches. During the forward pass the nodes store information about the actual values and the adjoints.

First we let $x\in\mathbb{R}^2$ represent the input $x=(a,b)$ such that

$$ \bar{x} \leftarrow \bar{u_1} \leftarrow \bar{u_2} \leftarrow \bar{d} $$
where $\bar{d}=1$. Work through this example and find the gradient $\bar{x} = \nabla_x d$ using the graph. On the forward pass $u_1=c=a+b=7$ and $u_2=ac=7\times4=28$ are stored alongside the adjoints

\begin{align*}
\bar{u_1}
    &= (\frac{\partial u_1}{\partial a}, \frac{\partial u_1}{\partial b})\\
    &= (1, 1)\\
\bar{u_2}
    &= (\frac{\partial u_2}{\partial a}, \frac{\partial u_2}{\partial c})\\
    &= (c, a)\\
    &= (7, 4)
\end{align*}

Therefore $\frac{\partial d}{\partial a} = \frac{\partial u_2}{\partial c}\frac{\partial u_1}{\partial a} + \frac{\partial u_2}{\partial a} = 4\times1+7=11, \frac{\partial d}{\partial b}= \frac{\partial u_2}{\partial c}\frac{\partial u_1}{\partial b} = 4\times1$ which means $\nabla d = (11, 4)$

Computing it directly using $d = a^2 + ab$
\begin{align*}
    \frac{\partial d}{\partial a} &= 2a + b = 11
    \\
    \frac{\partial d}{\partial b} &= a = 4
\end{align*}


## Exercise 2: Compute the partial derivatives automatically

Work through this simple example and implement a minimal version of a variable class and a method to compute gradients.

To do so we have created a `Variable` class that stores the value of the variable and the gradients with respect to its children.
The gradients are stored as a tuple of tuples, where each tuple contains a reference to the child variable and it correspoding partial
derivative.

Fill in the missing code in the `mul` function to compute the gradients of the variables with respect to their children.
Then fill in the missing code in the `compute_gradients` function to compute the gradients of the variables with respect to their children.

**a)**

In [2]:
from collections import defaultdict

class Variable:
    def __init__(self, value, gradients=None):
        self.value = value
        self._gradients = gradients if gradients is not None else ((self, np.sign(value)),)
        self._stored_gradients = None

    @property
    def gradients(self):
        return compute_gradients(self)
    
def add(a, b):
    "Create the variable that results from adding two variables."
    value = a.value + b.value    
    gradients = (
        (a, 1),  # d/da(a+b) is 1
        (b, 1)   # d/db(a+b) is 1
    )
    return Variable(value, gradients)

def mul(a, b):
    "Create the variable that results from multiplying two variables."
    # ---> TODO: fill in the missing code <---
    value = ___
    gradients = (
        (a, ___), # d/da(ab) is ____
        (b, ___)  # d/db(ab) is ____
    )
    return Variable(value, gradients)

def compute_gradients(variable):
    """ Compute the first derivatives of `variable` 
    with respect to child variables.
    """
    gradients = defaultdict(lambda: 0)
    
    def _compute_gradients(variable, total_gradient):
        for child_variable, child_gradient in variable._gradients:
            # ---> TODO: fill in the missing code <---
            # "Multiply the edges of a path":
            gradient = ___
            # "Add together the different paths":
            gradients[child_variable] = ___

            # if the child variable only has itself as a gradient 
            # we have reached the end of recursion
            criteria = (
                len(child_variable._gradients) == 1 and 
                child_variable._gradients[0][0] is child_variable
            )
            if not criteria:
                # recurse through graph:
                _compute_gradients(child_variable, gradient)
    
    _compute_gradients(variable, total_gradient=1)
    # (total_gradient=1 is from `variable` differentiated w.r.t. itself)
    return gradients

<!-- --- begin solution of exercise --- -->
**Solution.**

In [3]:
from collections import defaultdict

class Variable:
    def __init__(self, value, gradients=None):
        self.value = value
        self._gradients = gradients if gradients is not None else ((self, np.sign(value)),)
        self._stored_gradients = None

    @property
    def gradients(self):
        if self._stored_gradients is None:
            self._stored_gradients = dict(compute_gradients(self))
        return self._stored_gradients
    
def add(a, b):
    "Create the variable that results from adding two variables."
    value = a.value + b.value    
    gradients = (
        (a, 1),  # d/da(a+b) is 1
        (b, 1)   # d/db(a+b) is 1
    )
    return Variable(value, gradients)

def mul(a, b):
    "Create the variable that results from multiplying two variables."
    value = a.value * b.value
    gradients = (
        (a, b.value), # d/da(ab) b.value
        (b, a.value)  # d/db(ab) is a.value
    )
    return Variable(value, gradients)

def compute_gradients(variable):
    """ Compute the first derivatives of `variable` 
    with respect to child variables.
    """
    gradients = defaultdict(lambda: 0)
    
    def _compute_gradients(variable, total_gradient):
        for child_variable, child_gradient in variable._gradients:
            # "Multiply the edges of a path":
            gradient = total_gradient * child_gradient
            # "Add together the different paths":
            gradients[child_variable] += gradient
            # if the child variable only has itself as a gradient 
            # we have reached the end of recursion
            criteria = (
                len(child_variable._gradients) == 1 and 
                child_variable._gradients[0][0] is child_variable
            )
            if not criteria:
                # recurse through graph:
                _compute_gradients(child_variable, gradient)
    
    _compute_gradients(variable, total_gradient=1)
    # (total_gradient=1 is from `variable` differentiated w.r.t. itself)
    return gradients

<!-- --- end solution of exercise --- -->

**b)**
Test function against true value

In [4]:
a = Variable(4)
# TODO: fill in the missing code
b = ___
c = ___
d = ___

assert d.gradients[a] == ___

<!-- --- begin solution of exercise --- -->
**Solution.**

In [5]:
a = Variable(4)
b = Variable(3)
c = add(a, b)
d = mul(a, c)

assert d.gradients[a] == 11

<!-- --- end solution of exercise --- -->

## Linear rate neurons

Starting with the loss $\mathcal{L}=\frac{1}{2}(y - \hat{y})^2$ we have the computation graph in the figure below.

<!-- dom:FIGURE: [figures/auto-diff-graph.png] <div id="fig:auto-diff-graph"></div> -->
<!-- begin figure -->
<div id="fig:auto-diff-graph"></div>

<img src="figures/auto-diff-graph.png" ><p style="font-size: 0.9em"><i>Figure 2</i></p>
<!-- end figure -->

## Exercise 3: Validate the gradients

In this exercise we want to show the benefit of autodiff. 
In the previous module: LinearRegression, we saw that we had to compute gradients of the function manually - this is what autodiff is gonna automate for us. 
We will simply define the forward pass of our function (**fill in the blanks `___` in the functions `loss`, `sigma` and `y_hat` below**).
In this exercise we will use a predefined Variable class from `variable.py` to automatically compute its gradient. 
We will compare with the ground truth analytical gradient (we restrict ourselves to compare the partial derivative of one variable $w_1$).

In [6]:
# import a "complete" version of the above Variable class.
from variable import *

def loss(y, y_hat):
    return ___ # <---- FILL IN

def sigma(x):
    return ___ # <---- FILL IN

def y_hat(w_0, w_1, x_1):
    return ___ # <---- FILL IN

def dy_hat_dw_1(w_0, w_1, x_1):
    return x_1*y_hat(w_0, w_1, x_1) * (1 - y_hat(w_0, w_1, x_1))

def dloss_dw_1(y, w_0, w_1, x_1):
    return (y_hat(w_0, w_1, x_1) - y) * dy_hat_dw_1(w_0, w_1, x_1)

x_1 = Variable(0.1, name='x_1')
w_0 = Variable(4, name='w_0')
w_1 = Variable(3, name='w_1')
y = Variable(10, name='y')

def isclose(a, b):
    """
    Checks if two variables are similiar
    """
    a = a if not isinstance(a, Variable) else a.value
    b = b if not isinstance(b, Variable) else b.value
    return np.isclose(a, b)

# Test if analytic and autodiff for computing d_loss/d_w_1 is similar
assert isclose(loss(y, y_hat(w_0, w_1, x_1)).gradients[w_1], dloss_dw_1(y, w_0, w_1, x_1))
print('success!!!!!')

<!-- --- begin solution of exercise --- -->
**Solution.**

In [7]:
# import a "complete" version of the above Variable class.
from variable import *

def loss(y, y_hat):
    return 0.5 * (y - y_hat)**2

def sigma(x):
    return 1. / (1. + exp(-x))

def y_hat(w_0, w_1, x_1):
    return sigma(w_0 + w_1*x_1)

def dy_hat_dw_1(w_0, w_1, x_1):
    return x_1*y_hat(w_0, w_1, x_1) * (1 - y_hat(w_0, w_1, x_1))

def dloss_dw_1(y, w_0, w_1, x_1):
    return (y_hat(w_0, w_1, x_1) - y) * dy_hat_dw_1(w_0, w_1, x_1)

x_1 = Variable(0.1, name='x_1')
w_0 = Variable(4, name='w_0')
w_1 = Variable(3, name='w_1')
y = Variable(10, name='y')

def isclose(a, b):
    """
    Checks if two variables are similiar
    """
    a = a if not isinstance(a, Variable) else a.value
    b = b if not isinstance(b, Variable) else b.value
    return np.isclose(a, b)

# Test if analytic and autodiff for computing d_loss/d_w_1 is similar
assert isclose(loss(y, y_hat(w_0, w_1, x_1)).gradients[w_1], dloss_dw_1(y, w_0, w_1, x_1))
print('success!!!!!')

<!-- --- end solution of exercise --- -->